# Select Solar Cell Materials
Here, we perform a more advanced materials selection than the one outlined in `evaluate-meredig-predictions`. Now, we incorporate both the predicted stability and band gap energy to find materials that are the most likely to be both stable and have a favorable band gap energy

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import rankdata

## Load in the Predictions
The previous notebook saved them in `meredig_bandgap_predictions.prop`

In [2]:
results = pd.read_csv('meredig_bandgap_predictions.prop')
print('Loaded %d predictions'%len(results))

Loaded 4532 predictions


In [3]:
results.head()

,Entry,current_known_FE_measured,current_known_FE_predicted,ML_stability_measured,ML_stability_predicted,heuristic_stability_measured,heuristic_stability_predicted,bandgap_measured,bandgap_predicted,Unnamed: 9
0,HPd5F9,-1.096714,None,0.425405,None,0.300007,None,None,1.079357,NaN
1,HPt5F9,-1.012836,None,0.241396,None,0.270033,None,None,0.132911,NaN
2,HBr6F7,-0.737588,None,0.359994,None,0.221127,None,None,4.129982,NaN
3,H3NF6,-0.904819,None,0.297204,None,0.216766,None,None,6.658048,NaN
4,HSeF,-1.005354,None,0.257798,None,0.307043,None,None,3.695595,NaN


This dataset contains many useful columns. First of all, we have the stability score from [Meredig *et al*](http://link.aps.org/doi/10.1103/PhysRevB.89.094104)  from both their machine learning model (`ML_stability_measured`) and their heuristic (`heuristic_stability_measured`). We also have the predicted band gap energy from our model (`bandgap_predicted`)

## Get Only Entries with Target Band Gap
Get entries that are predicted to have a band gap within the target range

In [4]:
results.query('bandgap_predicted >= 0.9 and bandgap_predicted <= 1.7', inplace=True)
print('Found %d entries with desired band gap'%len(results))

Found 325 entries with desired band gap


## Rank by Stability and Band Gap
Rank each predicted entry based on the average stability score and the distance of the band gap from the center of the target range (1.3 eV). In both metrics, a lower score is better

In [5]:
results['bandgap_distance'] = np.abs(results['bandgap_predicted'] - 1.3)

Rank by stability

In [6]:
results['stability_score'] = rankdata(results[['ML_stability_measured','heuristic_stability_measured']].apply(np.mean, axis=1))

In [7]:
print('Best entries by stability:')
results.sort_values('stability_score', ascending=True).head(5)[['Entry', 'ML_stability_measured', 'heuristic_stability_measured']]

Best entries by stability:


,Entry,ML_stability_measured,heuristic_stability_measured
2069,LaSb3S3,0.203818,0.208914
4080,ThTe4P4,0.209678,0.203684
3646,Lu2Te5S,0.205412,0.213897
458,AlSb7Cl6,0.217835,0.201813
962,Mn3.5HCl6,0.209531,0.210558


Rank by bandgap

In [8]:
results['bandgap_score'] = rankdata(results['bandgap_distance'])

In [9]:
print('Best entries by band gap:')
results.sort_values('bandgap_score', ascending=True).head(5)[['Entry','bandgap_predicted']]

Best entries by band gap:


,Entry,bandgap_predicted
575,K5MnF8,1.318964
657,Ca3PI9,1.343025
1399,Sr3I9N,1.346771
1843,Cs5MnF9,1.243032
1037,CoB2F9,1.380256


## Rank by Both Scores
Get the materials predicted to have good stability *and* band gap energies

In [10]:
results['combined_score'] = results[['bandgap_score','stability_score']].apply(np.mean, axis=1)

In [11]:
print('Best materials by both scores:')
results.sort_values('combined_score', ascending=True).head(10)[['Entry','bandgap_predicted']]

Best materials by both scores:


,Entry,bandgap_predicted
1037,CoB2F9,1.380256
3414,YbAs7Cl6,1.156973
3884,Tl3OsO3.5,1.078918
1920,Cs8CoSe5,1.074358
421,Mg6SiTe8,1.119002
4404,PuMn1.5F8.5,1.489667
3011,TbP3Se,1.069602
3349,TmBr6N,1.102033
1919,Cs8FeSe5,1.048822
2609,Pm5LiTe8,1.079399


We note that these materials are different than those reported in [Table 2](https://www.nature.com/articles/npjcompumats201628/tables/2) of our paper. The difference is due in part to differences in the version of Magpie used when originally performing this analysis, and the version included in the SI to the paper - highlighting the importance of provenance tracking tools!